In [1]:
import awkward
import numpy as np

In [2]:
array = awkward.fromiter([
    [1,2,3,4.5],
    [2.5, 7.1],
    [8.8, 9.9, 23.4, 8.9]
])

In [7]:
array_sin = np.sin(array)
array_cos = np.cos(array)

In [10]:
arrays = dict(
    orig=array,
    sin=array_sin,
    cos=array_cos
)

In [11]:
arrays

{'orig': <JaggedArray [[1.0 2.0 3.0 4.5] [2.5 7.1] [8.8 9.9 23.4 8.9]] at 0x7f6605752d68>,
 'sin': <JaggedArray [[0.8414709848078965 0.9092974268256817 0.1411200080598672 -0.977530117665097] [0.5984721441039565 0.7289690401258759] [0.5849171928917617 -0.45753589377532133 -0.9869155581206487 0.5010208564578846]] at 0x7f66056f0898>,
 'cos': <JaggedArray [[0.5403023058681398 -0.4161468365471424 -0.9899924966004454 -0.2107957994307797] [-0.8011436155469337 0.6845466664428066] [-0.811093014061656 -0.8891911526253609 -0.16123796432418885 -0.8654352092411123]] at 0x7f66056f0860>}

In [12]:
awkward.save('mytest', arrays, name='ff')

In [14]:
readout = awkward.load('mytest.awkd')

In [17]:
list(readout.keys())

['fforig', 'ffsin', 'ffcos']

In [21]:
print(*list(readout.values()), sep='\n')

[[1.0 2.0 3.0 4.5] [2.5 7.1] [8.8 9.9 23.4 8.9]]
[[0.8414709848078965 0.9092974268256817 0.1411200080598672 -0.977530117665097] [0.5984721441039565 0.7289690401258759] [0.5849171928917617 -0.45753589377532133 -0.9869155581206487 0.5010208564578846]]
[[0.5403023058681398 -0.4161468365471424 -0.9899924966004454 -0.2107957994307797] [-0.8011436155469337 0.6845466664428066] [-0.811093014061656 -0.8891911526253609 -0.16123796432418885 -0.8654352092411123]]


In [22]:
arrays_plus = dict(
    orig=array+1,
    sin=array_sin+1,
    cos=array_cos+1
)

In [23]:
arrays_plus

{'orig': <JaggedArray [[2.0 3.0 4.0 5.5] [3.5 8.1] [9.8 10.9 24.4 9.9]] at 0x7f66051e2588>,
 'sin': <JaggedArray [[1.8414709848078965 1.9092974268256817 1.1411200080598671 0.02246988233490299] [1.5984721441039564 1.728969040125876] [1.5849171928917616 0.5424641062246787 0.013084441879351316 1.5010208564578846]] at 0x7f66051e25c0>,
 'cos': <JaggedArray [[1.5403023058681398 0.5838531634528576 0.010007503399554585 0.7892042005692204] [0.1988563844530663 1.6845466664428066] [0.18890698593834399 0.11080884737463914 0.8387620356758112 0.13456479075888772]] at 0x7f66051e2208>}

In [25]:
awkward.hdf5(arrays)

AttributeError: 'dict' object has no attribute 'name'

In [1]:
import utils.histoHelpers as uhh
import utils.uprootHelpers as uuh
from utils.SignalDescription import SignalDescription
import numpy as np
import mvatrain.preprocessors as mpp
import importlib

In [2]:
sd = SignalDescription()
sfs_4mu = sd.files('4mu', grouped=False, nth=-1)

In [3]:
sfs_4mu[0]

'root://cmseos.fnal.gov//eos/uscms/store/group/lpcmetx/MCSIDM/ffNtuple/2018/CRAB_PrivateMC/SIDM_BsTo2DpTo4Mu_MBs-100_MDp-0p25_ctau-18p75/190405_000651/0000/ffNtuple_1.root'

In [4]:
def match(obj):
    genp4 = uuh.p4Array(obj.tree['gen_p4'])
    mDarkPhoton = obj.tree['gen_pid'].array()==32
    jetp4 = uuh.p4Array(obj.tree['pfjet_p4'])
    mGendpMatch, mJetMatch = uuh.MaskArraysFromMatching(genp4[mDarkPhoton], jetp4)
    return mJetMatch[obj.mHLT].flatten()

def neutralEmEnerygFraction(obj):
    t = obj.tree
    jetp4 = uuh.p4Array(t['pfjet_p4'])
    _res = (t['pfjet_neutralEmE'].array()+t['pfjet_neutralHadronE'].array())/jetp4.energy
    return _res[obj.mHLT].flatten()
    
def pickExtreme(ja, maxmin):
    _ja = np.abs(ja[~np.isnan(ja)])
    if maxmin == 'max':
        _res = _ja[_ja.argmax()]
    if maxmin == 'min':
        _res = _ja[_ja.argmin()]
    cnts = _res.count()
    res = np.array([0.]*len(_res), dtype='float32')
    singletonIdx, zerotonIdx = np.where(cnts==1), np.where(cnts==0)
    res[singletonIdx] = _res[singletonIdx]
    res[zerotonIdx] = np.nan
    return res

def maxd0OfJet(obj):
    t = obj.tree
    candTkd0 = uuh.NestNestObjArrayToJagged(t['pfjet_pfcand_tkD0'].array())[obj.mHLT].flatten()
    return pickExtreme(candTkd0, 'max')

def mind0OfJet(obj):
    t = obj.tree
    candTkd0 = uuh.NestNestObjArrayToJagged(t['pfjet_pfcand_tkD0'].array())[obj.mHLT].flatten()
    return pickExtreme(candTkd0, 'min')

def getpt(obj):
    return uuh.p4Array(obj.tree['pfjet_p4'])[obj.mHLT].pt.flatten()

def geteta(obj):
    return uuh.p4Array(obj.tree['pfjet_p4'])[obj.mHLT].eta.flatten()

def gettkIso(obj):
    return obj.tree['pfjet_tkIsolation'].array()[obj.mHLT].flatten()

def getpfIso(obj):
    return obj.tree['pfjet_pfIsolation'].array()[obj.mHLT].flatten()

_pickmethods = {
    'j_match': match,
    'j_pt': getpt,
    'j_eta': geteta,
    'j_nef': neutralEmEnerygFraction,
    'j_maxd0': maxd0OfJet,
    'j_mind0': mind0OfJet,
    'j_tkiso': gettkIso,
    'j_pfiso': getpfIso,
}

In [61]:
p = ffPicker(sfs_4mu[0], pickmethods=_pickmethods)

In [62]:
pickResult = p.pick()

In [63]:
pickResult

{'j_match': array([ True,  True,  True, ...,  True,  True,  True]),
 'j_pt': array([50.350803 , 35.145393 ,  6.9153905, ...,  6.7672257,  5.291464 ,
         5.05025  ], dtype=float32),
 'j_eta': array([-0.53674996,  1.1519103 ,  0.42891517, ..., -1.9098365 ,
        -0.71814   , -0.08655441], dtype=float32),
 'j_nef': array([0.03169854, 0.06634919, 1.        , ..., 0.09096105, 0.99999994,
        1.        ], dtype=float32),
 'j_maxd0': array([1.0172485 , 9.7470455 ,        nan, ..., 0.01551633,        nan,
               nan], dtype=float32),
 'j_mind0': array([1.0172485e+00, 3.5946052e+00,           nan, ..., 1.1463119e-03,
                  nan,           nan], dtype=float32),
 'j_tkiso': array([0.2006171 , 0.24901195, 1.        , ..., 0.6974016 , 1.        ,
        1.        ], dtype=float32),
 'j_pfiso': array([0.28458518, 0.43789557, 0.78197026, ..., 0.72970414, 0.8313656 ,
        0.750592  ], dtype=float32)}

In [64]:
[a.size for a in pickResult.values()]

[7426, 7426, 7426, 7426, 7426, 7426, 7426, 7426]

In [5]:
mp = mpp.ffMultiPicker(sfs_4mu, pickmethods=_pickmethods)

In [6]:
mPickResults = mp.pick()

In [7]:
[a.size for a in mPickResults.values()]

[1112655, 1112655, 1112655, 1112655, 1112655, 1112655, 1112655, 1112655]

In [10]:
awkward.save('sig4mu.awkd', mPickResults, mode='w')

In [23]:
pickresreadout = awkward.load('sig4mu.awkd')

In [28]:
pickresreadout['j_match'].size

1112655

In [2]:
testarray = np.array([1,4,2])
ta = np.array([1,4,np.nan])
ta2 = np.array([])
tja = awkward.fromiter([
    [1,2,3,4.5],
    [2.5, 7.1],
    [],
    [8.8, 9.9, 23.4, 8.9]
])

In [20]:
print(repr(tja))

<JaggedArray [[1.0 2.0 3.0 4.5] [2.5 7.1] [] [8.8 9.9 23.4 8.9]] at 0x7fcc04904390>


In [6]:
tjaidx = awkward.fromiter([[0], [0], [], [0]])

In [8]:
tjaidx2 = awkward.fromiter([[], [], [], []])

In [21]:
emptyja = awkward.fromiter([])

In [22]:
emptyja

array([], dtype=float64)

In [17]:
tjaidx2.size

4

In [9]:
tja[tjaidx2]

TypeError: jagged index must be boolean (mask) or integer (fancy indexing)

In [35]:
mv

<JaggedArray [[4.5] [7.1] [] [23.4]] at 0x7f8dd57f6470>

In [40]:
cnts = mv.count()

In [51]:
res = np.array([0.]*len(mv), dtype='float32')

In [43]:
trueidx, falseidx = np.where(cnts==1), np.where(cnts==0)

In [55]:
res[trueidx] = mv[trueidx]
res[falseidx] = np.nan

In [67]:
arr1 = np.array(range(5))
arr2 = np.array(range(3))
arr3 = np.array(range(4))

In [68]:
np.concatenate([arr1, arr2, arr3])

array([0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 3])